In [1]:
# 现在导入 codecarbon，它会从指定的路径中加载
import codecarbon
print(codecarbon.__file__)  # 打印出 codecarbon 模块的路径，确保它是从新路径加载的


d:\我的坚果云\毕业设计\codecarbon_CHN\codecarbon\__init__.py


In [3]:
from codecarbon.external.geography import GeoMetadata

g = GeoMetadata()           # 不传参数，内部会自动调用 IP 接口
area = g.get_grid_area()
print("国家:", g.country_name)
print("省份:", g.region)
print("经纬度:", g.latitude, g.longitude)
print("电网区域:", area)


[codecarbon INFO @ 15:51:39] [GeoMetadata] 匹配排放因子: 0.935 (OM)


国家: China
省份: shandong
经纬度: 36.6683 116.9972
电网区域: 华北


In [ ]:
import os
from codecarbon import EmissionsTracker

# 设置代理（如有需要）
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"
os.environ["ALL_PROXY"]   = "socks5h://127.0.0.1:7890"

# ===== 初始化碳排放追踪器 =====
tracker = EmissionsTracker(
    project_name="MNIST",
    output_dir='D:\\我的坚果云\\毕业设计\\codecarbon_CHN\\carbon_logs',
    measure_power_secs=1,
    tracking_mode="process",
    log_level="error",
    grid_emission_mode="om"  # 使用华北的 OM 排放因子
)

tracker.start()

# ===== MNIST 训练部分 =====
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 数据准备
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 简单模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# 训练设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 开始训练
epochs = 15
for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f}")

# ===== 停止碳排放追踪并输出结果 =====
emissions = tracker.stop()
print(f"MNIST 训练任务碳排放总计: {emissions:.6f} kg CO₂eq")


Epoch 1/15 | Loss: 218.7458
Epoch 3/15 | Loss: 43.2463
